In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
print(module_path)

/Users/revilooliver/Documents/quantum_computing/research/pauli_sandwitching/cut4mitigation


In [65]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy
import circuit_cutter
import mlrecon_methods as ml
import numpy as np
import mthree

import qiskit
import qiskit.providers.aer.noise as noise
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, total_counts, norm_dict
from vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation
from mlrecon_methods import run_circuits, collect_fragment_circuits, organize_tomography_data_from_list

In [4]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.22.2', 'qiskit-aer': '0.11.1', 'qiskit-ignis': '0.7.1', 'qiskit-ibmq-provider': '0.19.2', 'qiskit': '0.39.2', 'qiskit-nature': '0.3.0', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [5]:
numpy.set_printoptions(linewidth = 200)

qubits = 5

simulation_backend = "qasm_simulator"

seed = 0
print_circuits = True

In [6]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-ornl', group='ornl', project='chm185')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_montreal') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_kolkata') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_mumbai') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_guadalupe') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q-ornl', grou

In [7]:
device = provider.get_backend('ibmq_toronto')
device

<IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>

In [8]:
device.configuration().to_dict()['max_shots']

32000

In [9]:
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel
noise_model = NoiseModel.from_backend(device, gate_error = False, thermal_relaxation= False)
print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['measure']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
  Specific qubit errors: [('measure', (0,)), ('measure', (1,)), ('measure', (2,)), ('measure', (3,)), ('measure', (4,)), ('measure', (5,)), ('measure', (6,)), ('measure', (7,)), ('measure', (8,)), ('measure', (9,)), ('measure', (10,)), ('measure', (11,)), ('measure', (12,)), ('measure', (13,)), ('measure', (14,)), ('measure', (15,)), ('measure', (16,)), ('measure', (17,)), ('measure', (18,)), ('measure', (19,)), ('measure', (20,)), ('measure', (21,)), ('measure', (22,)), ('measure', (23,)), ('measure', (24,)), ('measure', (25,)), ('measure', (26,))]


In [10]:
noisy_simulator = provider.get_backend('ibmq_qasm_simulator')

In [11]:
front_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_front.qasm")

In [12]:
mid_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_mid.qasm")

In [13]:
end_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_end.qasm")

In [14]:
orign_circuit = front_layer + mid_layer + end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_69775/743480212.py:1: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  orign_circuit = front_layer + mid_layer + end_layer


In [15]:
orign_circuit.draw()

┌───┐      ┌───────────────┐   ┌────────────────┐                 »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─────────────────»
           ├───┤      ├───────────────┤ │ └────────────────┘┌───────────────┐»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────────────────────────■─────────────────■────────»
     ├───────────────┤                                              │        »
q_3: ┤ Ry(-0.019851) ├──────────────────────────────────────────────■────────»
     └───────────────┘                                                       »
q_4: ────────────────────────────────────────────────────────────────────────»
                                                                             »
«                      
«q_0: ─────────────────
«                      
«q_1: ─────────────────
«     ┌───────────────┐
«q_2: ┤ Ry(0.0036205) ├
«     └┬──────────────┤
«q_3: ─┤ Ry(0.019873) ├
«      └──────────────┘
«q_4: ─────────────────
«

In [16]:
#checking circuit

In [17]:
def apply_checking_circuit(qc, ctrl_bits, ancilla_bits, side = None):
    if len(ctrl_bits) != len(ancilla_bits):
        print("Size mismatch")
        return None
    if side == 'front':
        for i in ancilla_bits:
            qc.h(i)
        for j,k in zip(ctrl_bits, ancilla_bits):
            qc.cz(j, k)
    elif side == 'end':
        for j,k in zip(ctrl_bits, ancilla_bits):
            qc.cz(j, k)
        for i in ancilla_bits:
            qc.h(i)
    else:
        print("Side undefined")

In [18]:
circuit = front_layer.copy()

In [19]:
apply_checking_circuit(circuit, [2], [4], side = 'front')
circuit += mid_layer
apply_checking_circuit(circuit, [2], [4], side = 'end')
circuit += end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_69775/2062351668.py:2: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += mid_layer
/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_69775/2062351668.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += end_layer


In [20]:
circuit.draw()

┌───┐      ┌───────────────┐   ┌────────────────┐                 »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─────────────────»
           ├───┤      ├───────────────┤ │ └────────────────┘┌───────────────┐»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────■───────────────────■─────────────────■────────»
     ├───────────────┤        │                                     │        »
q_3: ┤ Ry(-0.019851) ├────────┼─────────────────────────────────────■────────»
     └─────┬───┬─────┘        │                                              »
q_4: ──────┤ H ├──────────────■──────────────────────────────────────────────»
           └───┘                                                             »
«                         
«q_0: ────────────────────
«                         
«q_1: ────────────────────
«        ┌───────────────┐
«q_2: ─■─┤ Ry(0.0036205) ├
«      │ └┬──────────────┤
«q_3: ─┼──┤ Ry(0.019873) ├
«      │  └────┬───┬─────┘
«q_4: ─■───────┤ H ├──────
«              └───┘

In [21]:
pauli_list = read_from_file('LiH_hamiltonian.txt')
pauli_commute = find_commute_groups(pauli_list)

In [22]:
#generate individual measurment circuits that changes the basis
meas_qcs = []
for i in range(0, len(pauli_commute)):
    temp_qc = MeasureCircuit(pauli_commute[i], num_qubits = 4,num_qargs = 5)
    meas_qcs.append(temp_qc)
    print(temp_qc)

YYYY
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYZZ
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_2: ────────────────
                     
q_3: ────────────────
                     
q_4: ────────────────
                     
ZZYY
                     
q_0: ────────────────
                     
q_1: ────────────────
     ┌──────────────┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYZX
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_2: ────────────────
      ┌────────────┐ 
q_3: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_4: ────────────────
                     
ZXYY
       

In [23]:
circuit_list = []
for meas in meas_qcs:
    temp_circuit = circuit.copy()
    temp_circuit += meas
    circuit_list.append(temp_circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_69775/1717817760.py:4: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  temp_circuit += meas


In [24]:
circuit_list[-2].draw()

┌───┐      ┌───────────────┐   ┌────────────────┐  ┌────────────┐ »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├──┤ U(π/2,0,π) ├─»
           ├───┤      ├───────────────┤ │ └────────────────┘┌─┴────────────┴┐»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────■───────────────────■─────────────────■────────»
     ├───────────────┤        │                                     │        »
q_3: ┤ Ry(-0.019851) ├────────┼─────────────────────────────────────■────────»
     └─────┬───┬─────┘        │                                              »
q_4: ──────┤ H ├──────────────■──────────────────────────────────────────────»
           └───┘                                                             »
«                                                  
«q_0: ─────────────────────────────────────────────
«     ┌────────────┐                               
«q_1: ┤ U(π/2,0,π) ├───────────────────────────────
«     └────────────┘┌───────────────┐┌────────────┐
«q_2: ──────■───────┤ Ry(0.0036205) ├┤ U(π/2,0,π) ├
«           │       └┬──────────────┤├────────────┤
«q_3: ──────┼────────┤ Ry(0.019873) ├┤ U(π/2,0,π) ├
«           │        └────┬───┬─────┘└────────────┘
«q_4: ──────■─────────────┤ H ├────────────────────
«                         └───┘

In [25]:
# cuts = []
# for i in range(0, 1):
#     if i == 0 or i == qubits - 1:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 6)]
#     else:
#         cut = [(circuit.qubits[i], 5),(circuit.qubits[i], 4)]
#     cuts += cut

In [26]:
cuts = [(circuit.qubits[2], 2),(circuit.qubits[2], 4)]

In [27]:
fragments_list = []
wire_path_map_list = []
for temp_qc in circuit_list:
    print(temp_qc)
    fragments, wire_path_map = circuit_cutter.cut_circuit(temp_qc, cuts)
    fragments_list.append(fragments)
    wire_path_map_list.append(wire_path_map)

           ┌───┐      ┌───────────────┐   ┌────────────────┐ ┌──────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─┤ U(π/2,0,π/2) ├»
           ├───┤      ├───────────────┤ │ └────────────────┘┌┴──────────────┤»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────■───────────────────■─────────────────■────────»
     ├───────────────┤        │                                     │        »
q_3: ┤ Ry(-0.019851) ├────────┼─────────────────────────────────────■────────»
     └─────┬───┬─────┘        │                                              »
q_4: ──────┤ H ├──────────────■──────────────────────────────────────────────»
           └───┘                                                             »
«                                                      
«q_0: ─────────────────────────────────────────────────
«     ┌────────────

           ┌───┐      ┌───────────────┐   ┌────────────────┐ ┌──────────────┐»
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─┤ U(π/2,0,π/2) ├»
           ├───┤      ├───────────────┤ │ └────────────────┘┌┴──────────────┤»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────■───────────────────■─────────────────■────────»
     ├───────────────┤        │                                     │        »
q_3: ┤ Ry(-0.019851) ├────────┼─────────────────────────────────────■────────»
     └─────┬───┬─────┘        │                                              »
q_4: ──────┤ H ├──────────────■──────────────────────────────────────────────»
           └───┘                                                             »
«                                                    
«q_0: ───────────────────────────────────────────────
«     ┌──────────────┐ 

           ┌───┐      ┌───────────────┐   ┌────────────────┐                 »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─────────────────»
           ├───┤      ├───────────────┤ │ └────────────────┘┌───────────────┐»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────■───────────────────■─────────────────■────────»
     ├───────────────┤        │                                     │        »
q_3: ┤ Ry(-0.019851) ├────────┼─────────────────────────────────────■────────»
     └─────┬───┬─────┘        │                                              »
q_4: ──────┤ H ├──────────────■──────────────────────────────────────────────»
           └───┘                                                             »
«                                                  
«q_0: ─────────────────────────────────────────────
«     ┌────────────┐       

           ┌───┐      ┌───────────────┐   ┌────────────────┐                 »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─────────────────»
           ├───┤      ├───────────────┤ │ └────────────────┘┌───────────────┐»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────■───────────────────■─────────────────■────────»
     ├───────────────┤        │                                     │        »
q_3: ┤ Ry(-0.019851) ├────────┼─────────────────────────────────────■────────»
     └─────┬───┬─────┘        │                                              »
q_4: ──────┤ H ├──────────────■──────────────────────────────────────────────»
           └───┘                                                             »
«                                    
«q_0: ───────────────────────────────
«     ┌────────────┐                 
«q_1: ┤ U(π/2,0,π

In [28]:
print(wire_path_map_list[0])

{Qubit(QuantumRegister(5, 'q'), 0): ((0, Qubit(QuantumRegister(4, 'q'), 3)),), Qubit(QuantumRegister(5, 'q'), 1): ((0, Qubit(QuantumRegister(4, 'q'), 2)),), Qubit(QuantumRegister(5, 'q'), 2): ((1, Qubit(QuantumRegister(3, 'q'), 0)), (0, Qubit(QuantumRegister(4, 'q'), 1)), (1, Qubit(QuantumRegister(3, 'q'), 2))), Qubit(QuantumRegister(5, 'q'), 3): ((0, Qubit(QuantumRegister(4, 'q'), 0)),), Qubit(QuantumRegister(5, 'q'), 4): ((1, Qubit(QuantumRegister(3, 'q'), 1)),)}


In [29]:
total_variants = ml.fragment_variants(wire_path_map_list[0])

In [30]:
total_variants

24

In [31]:
shots = device.configuration().to_dict()['max_shots'] * total_variants

In [32]:
fragments_list[0][0].draw()

┌───────────────┐                                                       »
q_0: ┤ Ry(-0.019851) ├──────────────────────────────────────────────■────────»
     └───────────────┘                                              │        »
q_1: ─────────────────────────────────────────────■─────────────────■────────»
           ┌───┐      ┌───────────────┐           │         ┌───────────────┐»
q_2: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
           ├───┤      ├───────────────┤ │ ┌────────────────┐└┬──────────────┤»
q_3: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─┤ U(π/2,0,π/2) ├»
           └───┘      └───────────────┘   └────────────────┘ └──────────────┘»
«     ┌──────────────┐┌──────────────┐
«q_0: ┤ Ry(0.019873) ├┤ U(π/2,0,π/2) ├
«     └──────────────┘└──────────────┘
«q_1: ────────────────────────────────
«     ┌──────────────┐                
«q_2: ┤ U(π/2,0,π/2) ├────────────────
«     └──────────────┘                
«q_3: ────────────────────────────────
«

In [33]:
fragment_cuts_list = []
for index in range(0, len(fragments_list)):
    for idx, fragment in enumerate(fragments_list[index]):
        print(f"fragment {idx}:")
        print(fragment)
        print()
    fragment_cuts = ml.fragment_cuts(wire_path_map_list[index])
    fragment_cuts_list.append(fragment_cuts)


fragment 0:
     ┌───────────────┐                                                       »
q_0: ┤ Ry(-0.019851) ├──────────────────────────────────────────────■────────»
     └───────────────┘                                              │        »
q_1: ─────────────────────────────────────────────■─────────────────■────────»
           ┌───┐      ┌───────────────┐           │         ┌───────────────┐»
q_2: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
           ├───┤      ├───────────────┤ │ ┌────────────────┐└┬──────────────┤»
q_3: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─┤ U(π/2,0,π/2) ├»
           └───┘      └───────────────┘   └────────────────┘ └──────────────┘»
«     ┌──────────────┐┌──────────────┐
«q_0: ┤ Ry(0.019873) ├┤ U(π/2,0,π/2) ├
«     └──────────────┘└──────────────┘
«q_1: ────────────────────────────────
«     ┌──────────────┐                
«q_2: ┤ U(π/2,0,π/2) ├────────────────
«     └──────────────┘                
«q_3

In [34]:
print(fragment_cuts_list[-1])

[{'prep': 1, 'meas': 1}, {'prep': 1, 'meas': 1}]


In [35]:
wire_path_map

{Qubit(QuantumRegister(5, 'q'), 0): ((0, Qubit(QuantumRegister(4, 'q'), 3)),),
 Qubit(QuantumRegister(5, 'q'), 1): ((0, Qubit(QuantumRegister(4, 'q'), 2)),),
 Qubit(QuantumRegister(5, 'q'), 2): ((1, Qubit(QuantumRegister(3, 'q'), 0)),
  (0, Qubit(QuantumRegister(4, 'q'), 1)),
  (1, Qubit(QuantumRegister(3, 'q'), 2))),
 Qubit(QuantumRegister(5, 'q'), 3): ((0, Qubit(QuantumRegister(4, 'q'), 0)),),
 Qubit(QuantumRegister(5, 'q'), 4): ((1, Qubit(QuantumRegister(3, 'q'), 1)),)}

In [36]:
num_fragments = len(fragments)
num_fragments

2

In [37]:
num_tomo_circuits = len(meas_qcs)
num_tomo_circuits

25

In [38]:
fragments[0].draw()

┌───────────────┐                                                       »
q_0: ┤ Ry(-0.019851) ├──────────────────────────────────────────────■────────»
     └───────────────┘                                              │        »
q_1: ─────────────────────────────────────────────■─────────────────■────────»
           ┌───┐      ┌───────────────┐           │         ┌───────────────┐»
q_2: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
           ├───┤      ├───────────────┤ │ ┌────────────────┐└───────────────┘»
q_3: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─────────────────»
           └───┘      └───────────────┘   └────────────────┘                 »
«     ┌──────────────┐
«q_0: ┤ Ry(0.019873) ├
«     └──────────────┘
«q_1: ────────────────
«                     
«q_2: ────────────────
«                     
«q_3: ────────────────
«

In [39]:
fragments[1].draw()

┌───────────────┐                       
q_0: ┤ Ry(0.0038243) ├─■─────────────────────
     └─────┬───┬─────┘ │          ┌───┐      
q_1: ──────┤ H ├───────■──■───────┤ H ├──────
           └───┘          │ ┌─────┴───┴─────┐
q_2: ─────────────────────■─┤ Ry(0.0036205) ├
                            └───────────────┘

In [40]:
hardware_index = 0

In [41]:
tomography_circuits = []
for idx in range(0, len(fragments_list)):
    fragments = fragments_list[idx]
    wire_path_map = wire_path_map_list[idx]
    temp_frag_circuits = [0] * len(fragments)
    for i in range(0, len(fragments)):
        if i == hardware_index:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend, extra_qc = None)
        else:
            temp_frag_circuits[i] = collect_fragment_circuits(fragments[i], i, wire_path_map,
                                         shots = shots // total_variants,
                                         tomography_backend = simulation_backend)
    tomography_circuits.append(temp_frag_circuits)

In [42]:
#contains 25 basis
len(tomography_circuits)

25

In [43]:
#contains three fragments
len(tomography_circuits[0])

2

In [44]:
len(tomography_circuits[0][1])

1

In [45]:
#tomography circuits
len(tomography_circuits[0][1][0])

12

In [46]:
multi_exp_circuits = [[],[]]
for i in range(0, len(tomography_circuits)):
    #for each basis collect the three fragments:
    for j in range(0, num_fragments):
        fragment_circ = tomography_circuits[i][j][0]
        multi_exp_circuits[j] += fragment_circ

In [47]:
multi_exp_circuits[0][-200].draw()

░ ┌───────────────┐                    »
  q_0: ────────────────────░─┤ Ry(-0.019851) ├────────────────────»
       ┌─────────────────┐ ░ └───────────────┘                    »
  q_1: ┤ U3(-1.9106,π,0) ├─░──────────────────────────────────────»
       └─────────────────┘ ░       ┌───┐      ┌───────────────┐   »
  q_2: ────────────────────░───────┤ X ├──────┤ Ry(-0.015869) ├─■─»
                           ░       ├───┤      ├───────────────┤ │ »
  q_3: ────────────────────░───────┤ X ├──────┤ Ry(-0.003534) ├─■─»
                           ░       └───┘      └───────────────┘   »
c16_0: ═══════════════════════════════════════════════════════════»
                                                                  »
c16_1: ═══════════════════════════════════════════════════════════»
                                                                  »
c16_2: ═══════════════════════════════════════════════════════════»
                                                                  »
c16_3: ═══════════════════════════════════════════════════════════»
                                                                  »
«                                          ┌──────────────┐┌────────────┐ ░ ┌─┐»
«  q_0: ──────────────────────────■────────┤ Ry(0.019873) ├┤ U(π/2,0,π) ├─░─┤M├»
«                                 │        └──────────────┘└────────────┘ ░ └╥┘»
«  q_1: ────────■─────────────────■───────────────────────────────────────░──╫─»
«               │         ┌───────────────┐┌──────────────┐               ░  ║ »
«  q_2: ────────■─────────┤ Ry(-0.015751) ├┤ U(π/2,0,π/2) ├───────────────░──╫─»
«       ┌────────────────┐└┬──────────────┤└──────────────┘               ░  ║ »
«  q_3: ┤ Ry(-0.0036834) ├─┤ U(π/2,0,π/2) ├───────────────────────────────░──╫─»
«       └────────────────┘ └──────────────┘                               ░  ║ »
«c16_0: ═════════════════════════════════════════════════════════════════════╩═»
«                                                                              »
«c16_1: ═══════════════════════════════════════════════════════════════════════»
«                                                                              »
«c16_2: ═══════════════════════════════════════════════════════════════════════»
«                                                                              »
«c16_3: ═══════════════════════════════════════════════════════════════════════»
«                                                                              »
«                  
«  q_0: ───────────
«       ┌───┐   ┌─┐
«  q_1: ┤ H ├───┤M├
«       └┬─┬┘   └╥┘
«  q_2: ─┤M├─────╫─
«        └╥┘ ┌─┐ ║ 
«  q_3: ──╫──┤M├─╫─
«         ║  └╥┘ ║ 
«c16_0: ══╬═══╬══╬═
«         ║   ║  ║ 
«c16_1: ══╬═══╬══╩═
«         ║   ║    
«c16_2: ══╩═══╬════
«             ║    
«c16_3: ══════╩════
«

In [48]:
frag_datas = [0] * num_fragments
for i in range(0, num_fragments):
    if i == hardware_index:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = [8,11,14,16], 
                                     backend = device, shots = shots // total_variants, 
                                     optimization_level = 3, monitor_jobs = True)
    else:
        frag_datas[i] = run_circuits(multi_exp_circuits[i], initial_layout = None,
                                     backend = simulation_backend, shots = shots // total_variants,
                                     optimization_level = 3, monitor_jobs = True, noise_model = None)
       

job_id:  63777f8e18d74823c5f75526
Job Status: job is queued (15)    

IBMQJobApiError: '"HTTPSConnectionPool(host=\'api-qcon.quantum-computing.ibm.com\', port=443): Max retries exceeded with url: /api/Network/ibm-q-ornl/Groups/ornl/Projects/chm185/Jobs/63777f8e18d74823c5f75526/status/v/1 (Caused by NewConnectionError(\'<urllib3.connection.HTTPSConnection object at 0x7fe4ea506250>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known\'))"'

In [49]:
frag_datas[0] = [device.retrieve_job(job_id='63777f8e18d74823c5f75526').result()]

In [50]:
frag_datas[1] = run_circuits(multi_exp_circuits[1], initial_layout = None,
                             backend = simulation_backend, shots = shots // total_variants,
                             optimization_level = 3, monitor_jobs = True, noise_model = None)

job_id:  c14b8980-fc28-426d-98f2-ef269ee4483b
Job Status: job has successfully run


In [51]:
frag_datas[0][0].results[0].header.name

"(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z'))"

In [52]:
num_tomo_circuits

25

In [53]:
test = [[], []]

In [54]:
results_list = [[], []]
counts_list = [[], []] 
for i in range(0, num_fragments): 
    for j in range(0, num_tomo_circuits):
        start_idx = j * 12
        end_idx = (j + 1) * 12
        results_list[i].append(frag_datas[i][0].results[start_idx : end_idx])
        counts_list[i].append(frag_datas[i][0].get_counts()[start_idx : end_idx])


In [55]:
frag_datas[0][0].get_counts()

[{'0000': 3766,
  '0001': 4184,
  '0010': 56,
  '0011': 53,
  '0100': 3813,
  '0101': 4229,
  '0110': 57,
  '0111': 69,
  '1000': 3652,
  '1001': 4116,
  '1010': 52,
  '1011': 42,
  '1100': 3598,
  '1101': 4202,
  '1110': 56,
  '1111': 55},
 {'0000': 1940,
  '0001': 2171,
  '0010': 1817,
  '0011': 2065,
  '0100': 1911,
  '0101': 2343,
  '0110': 1954,
  '0111': 2160,
  '1000': 1932,
  '1001': 2062,
  '1010': 1819,
  '1011': 2006,
  '1100': 1868,
  '1101': 2118,
  '1110': 1763,
  '1111': 2071},
 {'0000': 2057,
  '0001': 2233,
  '0010': 1774,
  '0011': 2049,
  '0100': 1962,
  '0101': 2314,
  '0110': 1873,
  '0111': 2187,
  '1000': 1966,
  '1001': 2097,
  '1010': 1788,
  '1011': 1921,
  '1100': 1829,
  '1101': 2154,
  '1110': 1769,
  '1111': 2027},
 {'0000': 1438,
  '0001': 1703,
  '0010': 3111,
  '0011': 2585,
  '0100': 1156,
  '0101': 1320,
  '0110': 3235,
  '0111': 2481,
  '1000': 1247,
  '1001': 1454,
  '1010': 2795,
  '1011': 2193,
  '1100': 1057,
  '1101': 1216,
  '1110': 2831,
  '11

In [56]:
frag_datas[0][0].get_counts()

[{'0000': 3766,
  '0001': 4184,
  '0010': 56,
  '0011': 53,
  '0100': 3813,
  '0101': 4229,
  '0110': 57,
  '0111': 69,
  '1000': 3652,
  '1001': 4116,
  '1010': 52,
  '1011': 42,
  '1100': 3598,
  '1101': 4202,
  '1110': 56,
  '1111': 55},
 {'0000': 1940,
  '0001': 2171,
  '0010': 1817,
  '0011': 2065,
  '0100': 1911,
  '0101': 2343,
  '0110': 1954,
  '0111': 2160,
  '1000': 1932,
  '1001': 2062,
  '1010': 1819,
  '1011': 2006,
  '1100': 1868,
  '1101': 2118,
  '1110': 1763,
  '1111': 2071},
 {'0000': 2057,
  '0001': 2233,
  '0010': 1774,
  '0011': 2049,
  '0100': 1962,
  '0101': 2314,
  '0110': 1873,
  '0111': 2187,
  '1000': 1966,
  '1001': 2097,
  '1010': 1788,
  '1011': 1921,
  '1100': 1829,
  '1101': 2154,
  '1110': 1769,
  '1111': 2027},
 {'0000': 1438,
  '0001': 1703,
  '0010': 3111,
  '0011': 2585,
  '0100': 1156,
  '0101': 1320,
  '0110': 3235,
  '0111': 2481,
  '1000': 1247,
  '1001': 1454,
  '1010': 2795,
  '1011': 2193,
  '1100': 1057,
  '1101': 1216,
  '1110': 2831,
  '11

In [57]:
from mlrecon_methods import organize_tomography_data_from_list
organized_data = []
for i in range(0, num_tomo_circuits):
    frag_data = []
    frag_targets = ml.identify_frag_targets(wire_path_map_list[i])
    for j in range(0, num_fragments):
        frag_data.append(organize_tomography_data_from_list(results_list[j][i], counts_list[j][i],
                                              frag_targets[j].get("prep"),
                                              frag_targets[j].get("meas"),
                                              prep_basis = "SIC", extra_qc = []))
    organized_data.append(frag_data)

(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0') ('Z', 'Z', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z')) ('S0',) ('Z',) [1] [1]
(('S0', 'S0', 'S0', 'S0'), ('Z', 'X', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0') ('Z', 'X', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0'), ('Z', 'X', 'Z', 'Z')) ('S0',) ('X',) [1] [1]
(('S0', 'S0', 'S0', 'S0'), ('Z', 'Y', 'Z', 'Z')) ('S0', 'S0', 'S0', 'S0') ('Z', 'Y', 'Z', 'Z')
(('S0', 'S0', 'S0', 'S0'), ('Z', 'Y', 'Z', 'Z')) ('S0',) ('Y',) [1] [1]
(('S0', 'S1', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z')) ('S0', 'S1', 'S0', 'S0') ('Z', 'Z', 'Z', 'Z')
(('S0', 'S1', 'S0', 'S0'), ('Z', 'Z', 'Z', 'Z')) ('S1',) ('Z',) [1] [1]
(('S0', 'S1', 'S0', 'S0'), ('Z', 'X', 'Z', 'Z')) ('S0', 'S1', 'S0', 'S0') ('Z', 'X', 'Z', 'Z')
(('S0', 'S1', 'S0', 'S0'), ('Z', 'X', 'Z', 'Z')) ('S1',) ('X',) [1] [1]
(('S0', 'S1', 'S0', 'S0'), ('Z', 'Y', 'Z', 'Z')) ('S0', 'S1', 'S0', 'S0') ('Z', 'Y', 'Z', 'Z')
(('S0', 'S1', 'S0', 'S0'), ('Z', 'Y', 'Z', 'Z')) ('S1',) ('Y',) [1] [1

In [58]:
for final_bits, fixed_bit_data in frag_data[0].items():
    prep_meas_states, state_counts = zip(*fixed_bit_data.items())
    prep_labels, meas_labels = zip(*prep_meas_states)
    prep_qubit_num = len(prep_labels[0])
    meas_qubit_num = len(meas_labels[0])
    print(prep_qubit_num)

1
1
1
1
1
1
1
1


In [59]:
len(organized_data)

25

In [60]:
organized_data[2]

[{'000': {(('S0',), ('Zp',)): 13,
   (('S0',), ('Zm',)): 1,
   (('S0',), ('Xp',)): 14,
   (('S0',), ('Xm',)): 18,
   (('S0',), ('Yp',)): 12,
   (('S0',), ('Ym',)): 11,
   (('S1',), ('Zp',)): 5,
   (('S1',), ('Zm',)): 13,
   (('S1',), ('Xp',)): 6,
   (('S1',), ('Xm',)): 16,
   (('S1',), ('Yp',)): 10,
   (('S1',), ('Ym',)): 7,
   (('S2',), ('Zp',)): 8,
   (('S2',), ('Zm',)): 9,
   (('S2',), ('Xp',)): 13,
   (('S2',), ('Xm',)): 5,
   (('S2',), ('Yp',)): 13,
   (('S2',), ('Ym',)): 10,
   (('S3',), ('Zp',)): 5,
   (('S3',), ('Zm',)): 9,
   (('S3',), ('Xp',)): 21,
   (('S3',), ('Xm',)): 12,
   (('S3',), ('Yp',)): 11,
   (('S3',), ('Ym',)): 14},
  '001': {(('S0',), ('Zp',)): 16,
   (('S0',), ('Zm',)): 1,
   (('S0',), ('Xp',)): 14,
   (('S0',), ('Xm',)): 11,
   (('S0',), ('Yp',)): 9,
   (('S0',), ('Ym',)): 6,
   (('S1',), ('Zp',)): 6,
   (('S1',), ('Zm',)): 11,
   (('S1',), ('Xp',)): 6,
   (('S1',), ('Xm',)): 13,
   (('S1',), ('Yp',)): 10,
   (('S1',), ('Ym',)): 6,
   (('S2',), ('Zp',)): 12,
 

In [61]:
organized_data[-1][-1]

{'00': {(('S0',), ('Zp',)): 32000,
  (('S0',), ('Xp',)): 15953,
  (('S0',), ('Xm',)): 16047,
  (('S0',), ('Yp',)): 15905,
  (('S0',), ('Ym',)): 16095,
  (('S1',), ('Zp',)): 10724,
  (('S1',), ('Xm',)): 5211,
  (('S1',), ('Xp',)): 5319,
  (('S1',), ('Yp',)): 5372,
  (('S1',), ('Ym',)): 5265,
  (('S2',), ('Zp',)): 10558,
  (('S2',), ('Xm',)): 5443,
  (('S2',), ('Xp',)): 5392,
  (('S2',), ('Ym',)): 5423,
  (('S2',), ('Yp',)): 5343,
  (('S3',), ('Zp',)): 10594,
  (('S3',), ('Xm',)): 5405,
  (('S3',), ('Xp',)): 5297,
  (('S3',), ('Ym',)): 5275,
  (('S3',), ('Yp',)): 5305,
  (('S0',), ('Zm',)): 0,
  (('S1',), ('Zm',)): 0,
  (('S2',), ('Zm',)): 0,
  (('S3',), ('Zm',)): 0},
 '11': {(('S1',), ('Zp',)): 21276,
  (('S1',), ('Xp',)): 10822,
  (('S1',), ('Xm',)): 10648,
  (('S1',), ('Yp',)): 10649,
  (('S1',), ('Ym',)): 10714,
  (('S2',), ('Zp',)): 21440,
  (('S2',), ('Xp',)): 10579,
  (('S2',), ('Xm',)): 10586,
  (('S2',), ('Yp',)): 10585,
  (('S2',), ('Ym',)): 10649,
  (('S3',), ('Zp',)): 21405,


In [66]:
recombined_lists = []
filtered_recombined_lists = []
norm_filtered_lists = []
for i in range(0, num_tomo_circuits):
    direct_models = ml.direct_fragment_model(organized_data[i])
    likely_models = ml.maximum_likelihood_model(direct_models)

    direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map_list[i])
    likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map_list[i])
    recombined_lists.append(likely_recombined_dist)
    filtered_recombined_lists.append(filter_results(likely_recombined_dist, [0]))
    norm_filtered_lists.append(norm_dict(filter_results(likely_recombined_dist, [0])))

{'000': {(('S0',), ('Zp',)): 3766, (('S0',), ('Zm',)): 56, (('S0',), ('Xp',)): 1940, (('S0',), ('Xm',)): 1817, (('S0',), ('Yp',)): 2057, (('S0',), ('Ym',)): 1774, (('S1',), ('Zp',)): 1438, (('S1',), ('Zm',)): 3111, (('S1',), ('Xp',)): 360, (('S1',), ('Xm',)): 4252, (('S1',), ('Yp',)): 2242, (('S1',), ('Ym',)): 2380, (('S2',), ('Zp',)): 1421, (('S2',), ('Zm',)): 3325, (('S2',), ('Xp',)): 3470, (('S2',), ('Xm',)): 1263, (('S2',), ('Yp',)): 4035, (('S2',), ('Ym',)): 661, (('S3',), ('Zp',)): 1479, (('S3',), ('Zm',)): 3544, (('S3',), ('Xp',)): 3642, (('S3',), ('Xm',)): 1371, (('S3',), ('Yp',)): 825, (('S3',), ('Ym',)): 4358}, '001': {(('S0',), ('Zp',)): 4184, (('S0',), ('Zm',)): 53, (('S0',), ('Xp',)): 2171, (('S0',), ('Xm',)): 2065, (('S0',), ('Yp',)): 2233, (('S0',), ('Ym',)): 2049, (('S1',), ('Zp',)): 1703, (('S1',), ('Zm',)): 2585, (('S1',), ('Xp',)): 253, (('S1',), ('Xm',)): 3864, (('S1',), ('Yp',)): 2218, (('S1',), ('Ym',)): 1888, (('S2',), ('Zp',)): 1631, (('S2',), ('Zm',)): 2698, ((

{'10': {(('S0',), ('Zp',)): 15892, (('S0',), ('Xp',)): 7909, (('S0',), ('Xm',)): 8055, (('S0',), ('Yp',)): 7956, (('S0',), ('Ym',)): 7903, (('S1',), ('Zp',)): 5362, (('S1',), ('Xm',)): 2582, (('S1',), ('Xp',)): 2697, (('S1',), ('Ym',)): 2750, (('S1',), ('Yp',)): 2695, (('S2',), ('Zp',)): 5205, (('S2',), ('Xp',)): 2704, (('S2',), ('Xm',)): 2611, (('S2',), ('Yp',)): 2613, (('S2',), ('Ym',)): 2703, (('S3',), ('Zp',)): 5344, (('S3',), ('Xm',)): 2642, (('S3',), ('Xp',)): 2771, (('S3',), ('Yp',)): 2688, (('S3',), ('Ym',)): 2689, (('S0',), ('Zm',)): 0, (('S1',), ('Zm',)): 0, (('S2',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '00': {(('S0',), ('Zp',)): 16108, (('S0',), ('Xp',)): 8007, (('S0',), ('Xm',)): 8029, (('S0',), ('Yp',)): 8007, (('S0',), ('Ym',)): 8134, (('S1',), ('Zm',)): 1, (('S1',), ('Zp',)): 5280, (('S1',), ('Xp',)): 2734, (('S1',), ('Xm',)): 2617, (('S1',), ('Ym',)): 2704, (('S1',), ('Yp',)): 2632, (('S2',), ('Zp',)): 5335, (('S2',), ('Xm',)): 2682, (('S2',), ('Xp',)): 2611, (('S2',),

{'10': {(('S0',), ('Zp',)): 1, (('S2',), ('Zp',)): 1, (('S0',), ('Zm',)): 0, (('S0',), ('Xp',)): 0, (('S0',), ('Xm',)): 0, (('S0',), ('Yp',)): 0, (('S0',), ('Ym',)): 0, (('S1',), ('Zp',)): 0, (('S1',), ('Zm',)): 0, (('S1',), ('Xp',)): 0, (('S1',), ('Xm',)): 0, (('S1',), ('Yp',)): 0, (('S1',), ('Ym',)): 0, (('S2',), ('Zm',)): 0, (('S2',), ('Xp',)): 0, (('S2',), ('Xm',)): 0, (('S2',), ('Yp',)): 0, (('S2',), ('Ym',)): 0, (('S3',), ('Zp',)): 0, (('S3',), ('Zm',)): 0, (('S3',), ('Xp',)): 0, (('S3',), ('Xm',)): 0, (('S3',), ('Yp',)): 0, (('S3',), ('Ym',)): 0}, '00': {(('S0',), ('Zp',)): 31999, (('S0',), ('Xp',)): 15982, (('S0',), ('Xm',)): 16018, (('S0',), ('Yp',)): 15898, (('S0',), ('Ym',)): 16102, (('S1',), ('Zp',)): 10554, (('S1',), ('Xm',)): 5250, (('S1',), ('Xp',)): 5293, (('S1',), ('Ym',)): 5377, (('S1',), ('Yp',)): 5342, (('S2',), ('Zp',)): 10701, (('S2',), ('Xm',)): 5314, (('S2',), ('Xp',)): 5310, (('S2',), ('Ym',)): 5439, (('S2',), ('Yp',)): 5336, (('S3',), ('Zp',)): 10695, (('S3',)

{'10': {(('S0',), ('Zp',)): 15780, (('S0',), ('Xp',)): 7940, (('S0',), ('Xm',)): 8138, (('S0',), ('Yp',)): 8089, (('S0',), ('Ym',)): 7828, (('S1',), ('Zp',)): 5363, (('S1',), ('Xm',)): 2727, (('S1',), ('Xp',)): 2561, (('S1',), ('Ym',)): 2618, (('S1',), ('Yp',)): 2733, (('S2',), ('Zp',)): 5240, (('S2',), ('Xp',)): 2702, (('S2',), ('Xm',)): 2651, (('S2',), ('Ym',)): 2635, (('S2',), ('Yp',)): 2723, (('S3',), ('Zp',)): 5378, (('S3',), ('Xm',)): 2539, (('S3',), ('Xp',)): 2631, (('S3',), ('Ym',)): 2669, (('S3',), ('Yp',)): 2725, (('S0',), ('Zm',)): 0, (('S1',), ('Zm',)): 0, (('S2',), ('Zm',)): 0, (('S3',), ('Zm',)): 0}, '00': {(('S0',), ('Zp',)): 16220, (('S0',), ('Xm',)): 7957, (('S0',), ('Xp',)): 7965, (('S0',), ('Ym',)): 8031, (('S0',), ('Yp',)): 8052, (('S1',), ('Zp',)): 5346, (('S1',), ('Xp',)): 2666, (('S1',), ('Xm',)): 2685, (('S1',), ('Yp',)): 2705, (('S1',), ('Ym',)): 2681, (('S2',), ('Zp',)): 5418, (('S2',), ('Xm',)): 2649, (('S2',), ('Xp',)): 2707, (('S2',), ('Ym',)): 2672, (('S2'

In [67]:
# direct_models = ml.direct_fragment_model(frag_data)
# likely_models = ml.maximum_likelihood_model(direct_models)

# direct_recombined_dist = ml.recombine_fragment_models(direct_models, wire_path_map)
# likely_recombined_dist = ml.recombine_fragment_models(likely_models, wire_path_map)

In [68]:
direct_recombined_dist

{'00000': 0.0010722098098625002,
 '10100': 9.317311674465242e-05,
 '10000': 1.8678245686836613e-09,
 '00010': 0.03359098685802776,
 '10110': 0.0008056680693140736,
 '10010': -1.613798740836207e-08,
 '01010': 0.00031679992867104326,
 '11110': 3.1499908507745764e-05,
 '11010': -8.083276328184865e-10,
 '00001': 0.03035526450875671,
 '10101': 0.0012979936977685182,
 '10001': -1.3274233084694466e-08,
 '01001': 0.0003625100110245877,
 '11101': -4.797814367309859e-05,
 '11001': -2.3756036310560484e-09,
 '00011': 0.9178640899137732,
 '10111': 0.006078316718547895,
 '10011': -4.691046984689672e-07,
 '01011': 0.008371754600562173,
 '11111': -0.0002017642834406509,
 '11011': -2.5400080793186538e-08,
 '01000': 4.948781938942838e-06,
 '11100': 5.051801916764851e-06,
 '11000': -6.519646991395579e-11}

In [69]:
likely_recombined_dist

{'00010': 0.030625149711684303,
 '10110': 1.188891627981888e-05,
 '00001': 0.02707680779785602,
 '10101': 0.00012307849908345657,
 '00011': 0.9255189320125161,
 '10111': 0.011774815822200935,
 '01011': 0.00483500993843282,
 '11111': 3.431730194656798e-05}

In [70]:
filtered_recombined_lists

[{'0100': 0.05967214041862341,
  '0000': 0.058802177872159196,
  '1100': 0.0657880142101792,
  '1000': 0.06496784926397747,
  '0110': 0.059333216349922765,
  '0010': 0.05846222143273011,
  '1110': 0.06699258671629714,
  '1010': 0.06614399525990769,
  '0101': 0.05734858761652824,
  '0001': 0.05653275810471855,
  '1101': 0.06369143906658122,
  '1001': 0.06290004181945513,
  '0111': 0.05583902296206865,
  '0011': 0.055057799250609295,
  '1111': 0.06509017298788655,
  '1011': 0.06428420711194568},
 {'0000': 0.2545985522052221,
  '1000': 0.00028163468987396916,
  '0010': 0.2461144923561438,
  '1010': 0.0005405371640199364,
  '0001': 0.24472786965378002,
  '1001': 0.00017328238240321036,
  '0011': 0.2342042809346007,
  '1011': 0.0005482509200109264},
 {'0110': 0.0072149763819162535,
  '0010': 0.007321588190466553,
  '1110': 0.008462120313588156,
  '1010': 0.008576082022662928,
  '0101': 0.006371275651389137,
  '0001': 0.006445087374749802,
  '1101': 0.0066175182353584066,
  '1001': 0.0066922

In [71]:
from utils.utils import filter_results

In [72]:
filter_direct_recombined = filter_results(direct_recombined_dist, [0])

In [73]:
filter_direct_recombined

{'0000': 0.0010722098098625002,
 '0010': 0.03359098685802776,
 '1010': 0.00031679992867104326,
 '0001': 0.03035526450875671,
 '1001': 0.0003625100110245877,
 '0011': 0.9178640899137732,
 '1011': 0.008371754600562173,
 '1000': 4.948781938942838e-06}

In [74]:

filter_likely_recombined = filter_results(likely_recombined_dist, [0])

In [75]:
filter_likely_recombined

{'0010': 0.030625149711684303,
 '0001': 0.02707680779785602,
 '0011': 0.9255189320125161,
 '1011': 0.00483500993843282}

In [76]:
norm_filter_dist = norm_dict(filter_likely_recombined)

In [77]:
likely_recombined_dist['00011']

0.9255189320125161

In [78]:
#normalized distribution after mitigation: 0.9189375->0.9286796265616319
norm_filter_dist['0011']

0.9367070552565697

In [79]:
norm_filter_dist

{'0010': 0.030995361424800597,
 '0001': 0.027404125427155326,
 '0011': 0.9367070552565697,
 '1011': 0.004893457891474453}

In [80]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(norm_filtered_lists[i], shots = total_counts(norm_filtered_lists[i]), Pauli = Pauli_tuple[0])  

In [81]:
final_expect_val

(-1.0436437636872364+0j)

In [82]:
import json
with open('LiH_toronto_check2.json', 'w') as f:
    json.dump(norm_filtered_lists, f)